In [1]:
%pip install -U openmim
!mim install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%pip install mmdet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/mmdetection
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://git

In [5]:
import torch, torchvision
from mmdet.apis import init_detector, inference_detector
import mmcv
from mmcv.runner import load_checkpoint
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import draw_bounding_boxes
from torchvision import transforms
from PIL import Image
import sys
import os
from tqdm import trange
import json
from torchvision.ops import box_iou

/usr/local/lib/python3.7/dist-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


#VizDrone Dataset to COCO format

In [6]:
def get_bbox(path):
    translation_dct = {0:'ignored regions', 1:'pedestrian', 2:'people', 3:'bicycle', 4:'car', 
                      5:'van', 6:'truck', 7:'tricycle', 8:'awning-tricycle', 9:'bus', 10:'motor', 
                      11:'others'
                      }
    with open(path, 'r') as file:
        annotation = file.read().strip().split('\n')
        annotation_list=list(map(lambda x: list(map(int, x.split(',')[:6])), annotation))
        annotation_list = list(map(lambda a: [a[0], a[1], a[0]+a[2], a[1]+a[3]], list(filter(lambda a: a[5]==2, annotation_list))))
        return annotation_list

In [7]:
path_val='/content/drive/MyDrive/VisDrone2019-DET-val/'
path_train='/content/drive/MyDrive/VisDrone2019-DET-train/'
path_test='/content/drive/MyDrive/VIzDrone2019DET-test/'

```python
[
    {
        'filename': 'a.jpg',
        'width': 1280,
        'height': 720,
        'ann': {
            'bboxes': <np.ndarray> (n, 4),
            'labels': <np.ndarray> (n, ),
            'bboxes_ignore': <np.ndarray> (k, 4), (optional field)
            'labels_ignore': <np.ndarray> (k, 4) (optional field)
        }
    },
    ...
]
```

In [7]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
import numpy as np
from tqdm import tqdm

@DATASETS.register_module()
class VizdroneDataset(CustomDataset):
      CLASSES = ('Person', )

      def load_annotations(self, ann_file):
          cat2label = {k: i for i, k in enumerate(self.CLASSES)}
          image_list = os.listdir(f'{self.img_prefix}')
          data_infos = []
          for image_name in tqdm(image_list):
              img_path = f'{self.data_root}images/{image_name}'
              image = mmcv.imread(img_path)
              height, width = image.shape[:2]
    
              data_info = dict(filename=image_name, width=width, height=height)
              # load annotations
              label_prefix = f'{self.data_root}/annotations/'
              file_name = image_name.split('.')[0] + '.txt'
              bounding_boxes = np.array(get_bbox(f'{label_prefix}{file_name}'), 
                                dtype=np.float32).reshape(-1, 4)
              data_anno = dict(
                  bboxes=bounding_boxes,
                  labels=np.array([cat2label['Person'] 
                                   for _ in range(bounding_boxes.shape[0])], dtype=np.long),
                  #bboxes_ignore=np.array([], dtype=np.float32),
                  #labels_ignore=np.array([], dtype=np.long))
              )
              data_info.update(ann=data_anno)
              data_infos.append(data_info)
          return data_infos

create config file and replace it to /content/mmdetection/configs/vizdrone/

In [32]:
from mmcv import Config
cfg = Config.fromfile('./configs/dcn/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco.py')

In [40]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VizdroneDataset'
cfg.data_root = '/content/drive/MyDrive/'

cfg.data.test.type = 'VizdroneDataset'
cfg.data.test.data_root = '/content/drive/MyDrive/VIzDrone2019DET-test/'
#cfg.data.test.ann_file = 'train.txt'
cfg.data.test.img_prefix = 'images/'

cfg.data.train.type = 'VizdroneDataset'
cfg.data.train.data_root = '/content/drive/MyDrive/VisDrone2019-DET-train/'
cfg.data.train.pipeline = [
     {'type': 'LoadImageFromFile'},
     {'type': 'LoadAnnotations', 'with_bbox': True},
     {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
#     #{'type': 'Resize', 'img_scale': [(512, 512), (1024, 1024)], 'multiscale_mode': "range", 'keep_ratio': True}, #??? (check it)
     {'type': 'RandomFlip', 'flip_ratio': 0.5, 'direction': ['horizontal']},
     {'type': "Albu", 'transforms': [
         dict(type='RandomBrightnessContrast', brightness_limit=0.2,
              contrast_limit=0.2, p=0.5),
         dict(type='GaussNoise'),
         dict(type='ShiftScaleRotate', shift_limit=0.0625,
              scale_limit=0.1, rotate_limit=45, p=0.4),
     ],
      'bbox_params': 
         dict(
             type='BboxParams',
             format='pascal_voc',
             
             label_fields=['gt_labels'],
             min_visibility=0.0,
             filter_lost_elements=True),
         'keymap': {
         'img': 'image',
         'gt_bboxes': 'bboxes'
         }
    },
    #{'type': 'RandomCenterCropPad', 'crop_size': (512, 512),
    # 'ratios': (0.8, 0.9, 1.0, 1.1), 'test_pad_mode': None},
    {'type': 'CutOut', 'n_holes': (0, 3), 'cutout_ratio': (0.05, 0.2)},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'DefaultFormatBundle'},
    {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}
]
cfg.data.train.img_prefix = 'images/'

cfg.data.val.type = 'VizdroneDataset'
cfg.data.val.data_root = '/content/drive/MyDrive/VisDrone2019-DET-val/'
#cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'images/'

# modify num classes of the model in box head
for layer in cfg.model.roi_head.bbox_head:
    layer.num_classes = 1
cfg.load_from = '/content/drive/MyDrive/training_on_vizdrone/pretrained_on_vizdrone.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/training_on_vizdrone/epochs_from4/'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 12
cfg.lr_config.warmup_ratio = 1e-4
cfg.log_config.interval = 10
# cfg.custom_imports = dict(
#     imports=['mmdet.datasets.VizdroneDataset'],
#     allow_failed_imports=False)
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda'
cfg.runner.max_epochs = 3
# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet101'),
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, True, True, True)),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
     

In [41]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

# Create work_dir
#mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))


  0%|          | 0/6471 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
100%|██████████| 6471/6471 [42:39<00:00,  2.53it/s]
/content/mmdetection/mmdet/datasets/custom.py:182: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


In [19]:
cfg.dump('/content/configs.py')

In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-11-11 03:52:17,124 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
  0%|          | 0/548 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
100%|██████████| 548/548 [03:12<00:00,  2.84it/s]
2022-11-11 03:55:33,973 - mmdet - INFO - load checkpoint from local path: /content/drive/MyDrive/tra

In [23]:
cfg = Config.fromfile('/content/configs.py')

In [30]:
!python tools/test.py \
    /content/configs.py \
    /content/drive/MyDrive/training_on_vizdrone/pretrained_on_vizdrone.pth \
    --eval mAP --work-dir /content/metrics

/usr/local/lib/python3.7/dist-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
/content/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/content/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overload